<div style="padding:30px; color: white; background-color: #0071CD">
<center>
<img src="img/logoub.jpeg"></img>
<center>
<p>
<h1>Algorísmica Avançada</h1>
<h2>Pràctica 1 - Grafs </h2>
</center>
</p>
</div>

<div class="alert alert-danger" style="width:95%; margin:0 auto; padding">
<center><p><h2> ¡¡IMPORTANTE!! </h2></p> </center> 

<p>
Para la realizacón de esta práctica tendréis que utilizar vuestra propia clase `Graph` implementada en la Práctica 0. Si dicha clase no cumple los requisitos funcionales y no pasa los tests, con muy alta probabilidad los tests propuestos para esta práctica tampoco funcionen correctamente. 
</p>
</div>

<div class="alert alert-info">
<center>
  <h1>Introducción</h1>
</center>


A lo largo de esta práctica trabajaremos con el grafo generado a partir de la red de metro de Londres. En este grafo los nodos representan las estaciones y los ejes las vias que van de una estación a otra. Todos los ejes tienen cuatro atributos:

* Linea
* Color
* Nombre (de la linea)
* Distancia


In [3]:
'''
    He decidido usar la libreria de networkx ya que tras intentarlo con la que creé yo, no me funciona correctamente.
    Igualmente os voy a adjuntar el archivo de la libreria para que veais que almenos está realizada.
    Nombre del archivo: llibreriaGrafs
    
'''
from util import get_subway_graph, draw_subway_graph
import networkx as nx
G=nx.Graph()
G, lines = get_subway_graph('csv', nx.Graph)
G.edge[1][52]



{'color': '0d6928',
 'distance': 0.014782760229403606,
 'line': 4,
 'name': 'District Line',
 'stripe': 'NULL'}

Los nodos contienen el nombre de la estación, la latitud y longitud a la que está situada la estación, el número de lineas y la zona.

In [4]:
G.node[124]

{'display_name': 'NULL',
 'latitude': 51.5777,
 'longitude': -0.1458,
 'name': 'Highgate',
 'rail': '0',
 'total_lines': 1,
 'zone': 3.0}

Para mas consultas, la información ha sido extraida de Wikimedia Commons:

https://commons.wikimedia.org/wiki/London_Underground_geographic_maps/CSV

# util.py

En este archivo se os facilitan tres funciones que os permitiran cargar y visualizar la red de metro.
```python
"""
Retorna un objeto nx.Graph que corresponde al grafo de la red de metro y un 
diccionario con las lineas del metro
 - location: ruta donde esta almacenado el archivo .csv
 - Klass: la clase Graph que hemos desarrollado en la práctica 0
"""
G, lines = get_subway_graph(location, Klass)

"""
Dibuja el grafo que le pasemos por parametro.
- G: Grafo de la red de metro
- lines: diccionario con la información sobre las lineas del metro
- figsize: parametro opcional que nos permite definir el tamaño de la figura
- show_labels: parametro opcional que nos permite indicar si queremos mostrar los 
    nombres de las estaciones
"""
draw_metro_graph(G, lines, figsize=(10,6), show_labels=False)

```

In [5]:

#draw_subway_graph(G, lines, figsize=(10,6))

<div class="alert alert-info">
<center>
  <h1>Contenido</h1>
  </center><p>



<div class="alert alert-success" style="width:90%; margin:0 auto;">

  <h2><p>1- Dijkstra</p></h2>
  
  <p>
 Se propone que hagáis dos implementaciones del algoritmo <a href="https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm">Dijkstra</a>. 
 <ul><li>
 **dijkstra1:** La primera consiste en el algoritmo común en el que tendremos en cuenta las distancias definidas en los edges. 
 </li><li>
 **dijkstra2:** La segunda implementación es simplemente una ampliación del primer algoritmo en el que añadiremos una penalización por cambio de linea.</li>
 </ul>
</p>
<h3> Ejemplo _dijkstra2_</h3>
<p>
Para ilustrar lo que se pide en la práctica veamos como se calcularía el valor de **C** y **D** en el siguiente gráfico:
</p>


<p><center><img src='img/e1.png'></img></center></p>


<p>
<ul><li>
**[C]** Calculamos V(B) como V(A) + d([A,B]), calculamos V(C) como V(B) + d([B,C]), ya hemos acabado.
</li>
<li>
**[D]** Calculamos V(B) como V(A) + d([A,B]), calculamos V(D) como V(B) + d([B,D]), como [A,B] es linea azul y [B,D] es linea verde, añadimos una penalización por el transbordo a la que llamamos **P**. Por lo tanto, al final obtenemos que V(D) = V(B) + d([B,C]) + P
</li>
</ul>

<br>
-**V(X)**: Valor de X
<br>
-**d([X,Y])**: Distancia de X a Y
<br>
-**P**: Penalización por transbordo

</p>


<div class="alert alert-danger" style="width:80%; margin:0 auto; padding">
<center><p><h3> Código </h3></p> </center>
<p>
<h3>INPUT</h3>
<ul>
<li>__G__: Este es el grafo (en el caso de esta práctica la red de metro) que utilizaremos para buscar el camino. Debe de ser un objeto de tipo `Graph` como el que habéis implementado en la Práctica 0.</li>
<li>__origen__: Este parámetro corresponde al índice de un nodo. En este caso, como indexamos los nodos con el identificador de las paradas de Metro, deberá ser un entero _(e.g. 231)_.</li>
<li>__destino__: El indice del nodo al que queremos llegar.</li>
<li>__infinity=*(int)*__: Parametro opcional en el que definimos que numero nos va bien para utilizar como infinito en el momento de inicializar los pesos de los nodos.</li>
<li>__penalty=*(int)*__: (_Solo necesario para dijkstra2)_ Es un numero entero que corresponde a la penalización que aplicamos al valor de un nodo cuando hay cambio de linea.
</ul>
<br>
<h3>OUTPUT</h3>
El output de la función es un diccionario que contiene los siguientes valores:
<ul>
<li>__ _'path'_ __: Una lista de índices correspondientes al camino encontrado del nodo inicial al nodo final __ambos nodos, inicio y final, han de estar incluidos en esta lista__.</li>
<li>__ _'expanded'_ __: El numero de nodos que se han visitado para encontrar la solución.</li>
<li>__ _'distance'_ __: La distancia del camino mínimo desde el origen hasta el destino (es decir, el valor del nodo destino).
<ul>

</p>
</div>


In [12]:
def dijkstra(G, origen, destino, infinity=float('inf')):
    
    prev={} 
    dist={}
    #Los dos diccionarios de los previos y las distancias
    unvisited=[] #la cola donde ire metiendo los vecinos 
    visitats=[]  #lista de los nodos ya visitados
    expanded=0   #numero de expandidos
    
    for i in G.node:
        
        if i==origen:
            dist[i]=0
        prev[i]=None
        dist[i]=infinity
        #en este for, llenaremos la lista de prev y dist con None y inf respectivamente, menos al nodo origen que tendra dist=0
    unvisited.append([origen,0])
        #añadimos el origen a la cola
    
    while len(unvisited)!=0:
        
        minimo=get_min(unvisited)  #con la funcion get_min sacaremos el nodo con distancia minima de la cola

        if minimo[0] == destino:
            #si el valor minimo que sacamos es el destino, procederemos a construir el camino
            return construct_path(prev,destino,origen), expanded, dist[destino] 
        if minimo[0] not in visitats:
            visitats.append(minimo) #si el minimo no es visitado, lo añadiremos y se suma como expandido
            expanded+=1
            for i in G.neighbors(minimo[0]):
                if dist[i]>minimo[1]+ G.edge[i][minimo[0]].get("distance"):
                    dist[i]=minimo[1]+ G.edge[i][minimo[0]].get("distance")
                    prev[i]=minimo[0]
                    unvisited.append([i,dist[i]])
                    '''
                    inspeccionaremos los vecinos de minimo, a estos les actualizaremos la distancia y les cambiaremos de None
                    a minimo como su "padre"
                    posteriormente, añadiremos cada vecino a la cola de prioridad y cuando salgamos del for eliminaremos
                    el nodo minimo con el que estabámos trabajando.
                    
                    '''
        
            unvisited.remove(minimo)
        
                    
        
        

            

def get_min(unvisited,infinity=float('inf')):
    tmp=[None,infinity] #usamos un nodo temporal iniciandolo a None y infinito
    for i in unvisited:
        if i[1]<tmp[1]: #si la distancia de i es mas pequeña que tmp, se le cambiará
            tmp=[i[0],i[1]] #de esta manera podremos conseguir el nodo más pequeño
            
    return tmp

def construct_path(prev,destino,origen):
    path=[]
    aux=destino
    
    
    while prev[aux]!=origen:
        path.append(aux)
        
        aux=prev[aux]
    path.append(aux)
    path.append(origen)
    '''
    En esta función construiremos el camino, iremos recorriendo los prevs del destino al origen
    Para ello definimos un nodo auxiliar que lo inicializamos como destino. Con el while ponemos la condicion: mientras el padre
    de aux no sea origen, ve añadiendo el aux. Ese aux cambiara a su padre a cada iteración hasta que llegue al antepenultimo
    los cuales tenemos que añadir nosotros con un append. Finalmente invertimos el camino y ya lo tenemos de Origen-Destino
    '''
    
    path.reverse()
    
    return path 

    

In [13]:

dijkstra(G, 10, 235)


([10,
  128,
  39,
  145,
  89,
  277,
  192,
  107,
  133,
  146,
  236,
  99,
  74,
  17,
  110,
  265,
  1,
  73,
  182,
  194,
  5,
  252,
  251,
  235],
 279,
 0.31895111263175857)

In [8]:
def dijkstra2(G, origen, destino, penalty=5000000, infinity=float('inf')):
    '''
    Este algoritmo sigue la misma mecanica que dijkstra anterior pero vamos a vigilar los cambios de linea
    '''
    prev={}
    dist={}
    unvisited=[]
    visitats=[]
    expanded=0
    
    for i in G.node:
        
        if i==origen:
            dist[i]=0
        prev[i]=None
        dist[i]=infinity
    
    unvisited.append([origen,0])
    
    while len(unvisited)!=0:
        
        minimo=get_min(unvisited)

        if minimo[0] == destino:
            return construct_path(prev,destino,origen), expanded, dist[destino]
        if minimo[0] not in visitats:
            visitats.append(minimo)
            expanded+=1
            for i in G.neighbors(minimo[0]):
                if dist[i]>minimo[1]+ G.edge[i][minimo[0]].get("distance"):
                    #vamos a tener en cuenta los siguientes casos
                    if minimo[0]==origen or prev[i]==origen:
                        dist[i]=minimo[1]+ G.edge[i][minimo[0]].get("distance")
                        prev[i]=minimo[0]
                        unvisited.append([i,dist[i]])
                        '''
                        Si el minimo es el origen o su padre es el origen, significa que no ha habido cambio de linea
                        alguno, por eso, lo hacemos como en dijkstra 1
                        '''
                    
                    elif G.edge[minimo[0]][i].get("line") != G.edge[minimo[0]][prev[minimo[0]]].get("line"):
                        dist[i]=minimo[1]+ G.edge[i][minimo[0]].get("distance")+penalty
                        prev[i]=minimo[0]
                        unvisited.append([i,dist[i]])
                        '''
                        Aqui tenemos otro caso. Este nos compara si la linea que vamos a visitar en esta iteración
                        es diferente a la que hemos visitado anteriormente. Es decir comparamos dos aristas donde la comparación
                        se centra en su atributo line. Si no son la misma linea, añadiremos la penalty.
                        
                        '''
                    else:
                        dist[i]=minimo[1]+ G.edge[i][minimo[0]].get("distance")
                        prev[i]=minimo[0]
                        unvisited.append([i,dist[i]])
                        '''
                        Para todos los otros casos, vamos a hacer que la distancia se actualice normal como en dijkstra1
                        '''
                
        
            unvisited.remove(minimo)
        

                
                                

def get_min(unvisited,infinity=float('inf')):
    tmp=[None,infinity]
    for i in unvisited:
        if i[1]<tmp[1]:
            tmp=[i[0],i[1]]
            
        


    return tmp

def construct_path(prev,destino,origen):
    path=[]
    aux=destino
    
    
    while prev[aux]!=origen:
        path.append(aux)
        
        aux=prev[aux]
    path.append(aux)
    path.append(origen)
    
    path.reverse()
    
    return path   

    

In [9]:
dijkstra2(G, 10, 235)


([10,
  128,
  39,
  145,
  223,
  126,
  60,
  151,
  197,
  107,
  133,
  146,
  236,
  99,
  74,
  17,
  110,
  265,
  1,
  73,
  182,
  194,
  5,
  252,
  251,
  235],
 41,
 0.3249863215971557)

<div class="alert alert-warning" style="width:80%; margin:0 auto; padding">
<center><p><h3> Comentarios Dijkstra</h3></p> </center> </div>

Para esta práctica he usado la libreria de networkx, ya que la mia no me funcionaba correctamente. Los comentarios especificos del código estan comentados en el mismo para facilitar la lectura.

### Estructuras principales

La implementacion se ha basado en tener dos diccionarios uno que relacionara los nodos padre con sus hijos y otro que almacenaba las distancias de cada nodo. Para controlar los visitados he usado una lista donde añadia los nodos que habian sido visitados. 
Como este algoritmo requiere un heap o cola de prioridades, he hecho una lista donde en un cierto momento pasabámos esta "cola" por parametro de una función para obtener la distancia mínima y de esta manera, cumple su función. Esta cola, tenia dentro pequeñas listas con su nodo y distancia ([[nodo,distancia],[nodo2,distancia2]...]).

### Algoritmo

El algoritmo se basa en construir el camino entre el nodo origen y el nodo destino que nosotros le pasamos. De esta forma, si el nodo al que llamamos minimo (minimo en la cola en esa iteración) no es el destino, lo añadimos a visitados(si es no vistado) y haremos un bucle donde vamos a inspeccionar las distancias de todos los vecinos y a meter esos vecinos en nuestra cola. Al salir del bucle vamos a sacar el nodo minimo con el que hemos trabajado y vamos a volver a chequear que nodo es el que tiene distancia minima, y asi hasta que la longitud de la cola sea 0 o el minimo sea el nodo destino.

### Algoritmos auxiliares

He usado el get_min y el construct_path para coger el minimo de la cola y hacer el camino entre origen-->destino respectivamente

En el get_min uso un nodo temporal y itero en un for los nodos que hay en la cola, si hay alguno menor que el temporal, el temporal adquiere el valor de ese nodo, hasta acabar la iteración, que es cuando devuelve el nodo con menor distancia.

En el construct_path tendremos un nodo temporal que nos apuntara  al destino. Luego entramos en un bucle con la condicion de repetición que si el previo del aux es el origen, salga del while. A cada iteración vamos a ir añadiendo al camino el temporal, el cual cambiara a la posicion previa a cada iteración. Cuando salgamos acabamos de añadir el penultimo y último, revertimos la lista del camino y lo devolvemos.

### Complejidad

La complejidad de este algoritmo es O(|Nodos|+|Edges|*log(|Nodos|), dado que usamos una cola de prioridad, si no fuera así el coste seria mucho más alto, O(|Nodos^2|).



<div class="alert alert-warning" style="width:80%; margin:0 auto; padding">
<center><p><h3> Comentarios Dijkstra2</h3></p> </center> </div>

En esta segunda version de dijkstra con penalizaciones he hecho variaciones en el algoritmo que compara y define las distancias.
Las estructuras usadas, las funciones complementarias que uso y la complejidad es la misma que en dijkstra 1. Para no ser redundante explicando lo mismo, voy a dar la descripción de esta segunda version del algoritmo.

### Algoritmo Dijkstra 2 con penalties

En mi algoritmo defino 3 casos que pueden darse.
En uno de ellos el caso de que el nodo minimo, asi nombramos al que hemos sacado de la cola en esta iteración, sea el origen o sea el hijo del origen. Si es este caso, actualizaremos distancias como en dijkstra1

En el segundo caso es donde investigamos los cambios de lineas, compararemos la linea de la arista de nuestro minimo actual con el vecino de la iteracion del for, con la linea de la arista del minimo actual y el padre del minimo. Si esta resulta ser diferente, aplicaremos el penalty definido.

Para todos los otros casos, aplicaremos la actualizacion de distancia normal, como hemos hecho en dijkstra 1.



<div class="alert alert-success" style="width:90%; margin:0 auto;">

  <h2><p>2- Matriz de distancias</p></h2>
   <p>
 Se propone el desarrollo de un algoritmo que dado un grafo construya una matriz de $V \times V$ (donde $V$ es el número de vértices del grafo) que contenga la distancia mínima de cada vértice al resto. 
</p>

<p></p>

<p>
<b> Nota: </b> Recordad que en Algorítmica Avanzada buscamos la implementación de algoritmos que no solo resuelvan el problema, sino que lo hagan de manera eficiente
</p>


<div class="alert alert-danger" style="width:80%; margin:0 auto; padding">
<center><p><h3> Código </h3></p> </center>
<p>
<h3>INPUT</h3>
<ul>
<li>__G__: Este es el grafo (en el caso de esta práctica la red de metro) sobre el que calcularemos la matriz de distancias. Debe de ser un objeto de tipo `Graph` como el que habéis implementado en la Práctica 0.</li>
</ul>
<br>
<h3>OUTPUT</h3>
<ul>
<li>El output de la función es un diccionario cuya clave es un nodo (origen) y el valor es otro diccionario con la forma $\{nodo\_destino: distancia\}$.</li>
    </ul>

</p>
</div>


In [10]:
def dist_mat(G):
    
    matriz={} #Creamos el diccionario matriz
    for i in G.nodes():
        '''
        Por cada nodo del conjunto de nodos, haremos un update al diccionario matriz
        de la forma que nos piden para que sea una matriz de distancias.
        Con dijksta le pasaremos por parametro el nodo i, este representará el origen del grafo a esta iteración
        con la modificación que hemos hecho, dijkstra va a devolver la distancia del origen(i)--> todos los demás
        nodos.
        
        '''
        matriz.update({i:dijkstra_matriz(G,i)})
    
    
    return matriz
    
    
    
def dijkstra_matriz(G,origen,infinity=float('inf')): #igual que dijksta pero modificando partes
    prev={}
    dist={}
    unvisited=[]
    visitats=[]
    #no tendremos extended ni destino
    
    for i in G.node:
        
        if i==origen:
            dist[i]=0
        prev[i]=None
        dist[i]=infinity
    
    unvisited.append([origen,0])
    
    while len(unvisited)!=0:
        
        minimo=get_min(unvisited)
        
        #no tendremos condicion si el nodo minimo==destino devuelva el camino
        
        
        if minimo[0] not in visitats:
            visitats.append(minimo)
            for i in G.neighbors(minimo[0]):
                if dist[i]>minimo[1]+ G.edge[i][minimo[0]].get("distance"):
                    dist[i]=minimo[1]+ G.edge[i][minimo[0]].get("distance")
                    prev[i]=minimo[0]
                    unvisited.append([i,dist[i]])
                
        
            unvisited.remove(minimo)
    dist.update({origen:0})
    return dist
    #como último paso tendremos que devuelva la matriz de distancias   
                    
        
        

#no tendremos algoritmo que nos devuelva el camino           

def get_min(unvisited,infinity=float('inf')):
    tmp=[None,infinity]
    for i in unvisited:
        if i[1]<tmp[1]:
            tmp=[i[0],i[1]]
            
    return tmp
    
    
    
    
    

In [11]:
dist_mat(G)

{1: {1: 0,
  2: 0.2121536018358775,
  3: 0.2144982234992658,
  4: 0.2738273414306536,
  5: 0.04963783753701469,
  6: 0.4600942766465704,
  7: 0.19473965556078465,
  8: 0.20782075683849527,
  9: 0.29035873840021287,
  10: 0.21132669558431227,
  11: 0.14503838956708368,
  12: 0.2171615702214243,
  13: 0.19676363357353835,
  14: 0.20535801623806257,
  15: 0.37137230752791306,
  16: 0.41526336235398725,
  17: 0.06780056085904503,
  18: 0.11326599207611765,
  19: 0.37217067660757724,
  20: 0.34395106456086055,
  21: 0.4174651460943717,
  22: 0.2049834627148244,
  23: 0.22108597550112308,
  24: 0.23340901332418063,
  25: 0.1833224432143382,
  26: 0.2877099804953232,
  27: 0.27928611973473366,
  28: 0.15161819782330208,
  29: 0.19590089205662187,
  30: 0.04581639343365586,
  31: 0.2775552232578808,
  32: 0.2923019570804816,
  33: 0.26384734296867624,
  34: 0.2625605535286902,
  35: 0.1869883779576289,
  36: 0.27681846473974886,
  37: 0.3923890969185705,
  38: 0.31945285475067825,
  39: 0.1946

<div class="alert alert-warning" style="width:80%; margin:0 auto; padding">
<center><p><h3> Comentarios Matriz de Distancias</h3></p> </center> </div>



Como bien nos comentaron en problemas, hacer un algoritmo usando dos fors anidados era una idea tan sencilla como ineficiente(Me duró varios minutos a que me devolviese la matriz). 
Por eso he decidido implementar el algoritmo de la matriz de distancias modificando dijkstra1.
Donde en este nuevo dijkstra lo que devuelvo es el diccionario de las distancias y por ello recorto algunas partes del algoritmo original, la variable destino que nos indicaba el final del camino la elimino ya que el destino es el final y por ello no tiene mucha lógica. La otra parte es el construct_path, no me interesa para nada perder tiempo en saber el camino.

Su complejidad es la que tenemos en dijkstra1, O(|Nodos|+|Edges|*log(|Nodos|), este algoritmo tarda alrededor de 20 segundos, teniendo en cuenta que hay alrededor de 300 nodos. Comparandolo con la opción de fuerza bruta, es una mejora muy buena.



<div class="alert alert-info">
<center>
  <h1>Entrega</h1>
</center>
<p>
La entrega de esta práctica se podrá realizar en el campus virtual hasta el día __21 de Octubre a las 23:55__. En la tarea que se habilitará en el campus deberéis colgar __el archivo .ipynb__ asi como __el fichero .pyc con vuestra clase grafo__ con los nombres:
</p>
<p>
```
* AA_P1_<apellido1>_<apellido2>_<nombre>.ipynb
* graphs.pyc
```

</p>
<p>
Por ejemplo:
</p>
<p>
```
AA_P1_Doe_Ritchie_John.ipynb
```

</p>
<p>

Es fundamental que el código esté bién comentado y con un análisis de complejidad exhaustivo del algoritmo.
</p>
</div>